In [1]:
import sys
sys.path.append(r'D:\Fausto Stangler\Documentos\Python\DSH')
import assets.helper as b3
import assets.functions as run




In [2]:
import assets.helper as b4
import assets.functions as run4


In [67]:
import assets.helper as b3
import assets.functions as run



In [3]:
import os
import datetime
import time
import winsound
import random

import pandas as pd

from google.cloud import storage
import json
import gzip
import io
import base64

import requests
from bs4 import BeautifulSoup

import unidecode
import string
import re

### CVM IMPORT

In [151]:
df_rad_web = pd.read_parquet('datasets/rad.parquet')


In [152]:
df_rad_web

,companhia,trimestre,conta,descrição,valor,unidade,demo_det,demo_fin,url
0,2W ECOBANK SA,2019-12-31,0.01.01,Ações ON,113548407,UNIDADE,Ações,Ações ON,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...
1,2W ECOBANK SA,2019-12-31,0.01.02,Ações PN,0,UNIDADE,Ações,Ações PN,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...
2,2W ECOBANK SA,2019-12-31,0.02.01,Ações ON em Tesouraria,0,UNIDADE,Ações em Tesouraria,Ações ON,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...
3,2W ECOBANK SA,2019-12-31,0.02.02,Ações PN em Tesouraria,0,UNIDADE,Ações em Tesouraria,Ações PN,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...
4,2W ECOBANK SA,2019-12-31,1,Ativo Total,415000,UNIDADE,Balanço Patrimonial Ativo,DFs Individuais,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...
...,...,...,...,...,...,...,...,...,...
1037,2W ECOBANK SA,2020-03-31,7.08.04.02,Dividendos,0,UNIDADE,Demonstração de Valor Adicionado,DFs Consolidadas,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...
1038,2W ECOBANK SA,2020-03-31,7.08.04.03,Lucros Retidos / Prejuízo do Período,12000,UNIDADE,Demonstração de Valor Adicionado,DFs Consolidadas,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...
1039,2W ECOBANK SA,2020-03-31,7.08.04.04,Part. Não Controladores nos Lucros Retidos,0,UNIDADE,Demonstração de Valor Adicionado,DFs Consolidadas,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...
1040,2W ECOBANK SA,2020-03-31,7.08.05,Outros,0,UNIDADE,Demonstração de Valor Adicionado,DFs Consolidadas,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...


In [150]:
# Simulating the iteration process up to the first saving point, given size = 34286 and bin_size = 50
size = 155
bin_size = 50

# Simulate iterations
for i in range(size):
    print(i, (size - i - 1), (size - i - 1) % bin_size)
    if (size - i - 1) % bin_size == 0:
        # Condition met for saving
        print('save')



0 154 4
1 153 3
2 152 2
3 151 1
4 150 0
save
5 149 49
6 148 48
7 147 47
8 146 46
9 145 45
10 144 44
11 143 43
12 142 42
13 141 41
14 140 40
15 139 39
16 138 38
17 137 37
18 136 36
19 135 35
20 134 34
21 133 33
22 132 32
23 131 31
24 130 30
25 129 29
26 128 28
27 127 27
28 126 26
29 125 25
30 124 24
31 123 23
32 122 22
33 121 21
34 120 20
35 119 19
36 118 18
37 117 17
38 116 16
39 115 15
40 114 14
41 113 13
42 112 12
43 111 11
44 110 10
45 109 9
46 108 8
47 107 7
48 106 6
49 105 5
50 104 4
51 103 3
52 102 2
53 101 1
54 100 0
save
55 99 49
56 98 48
57 97 47
58 96 46
59 95 45
60 94 44
61 93 43
62 92 42
63 91 41
64 90 40
65 89 39
66 88 38
67 87 37
68 86 36
69 85 35
70 84 34
71 83 33
72 82 32
73 81 31
74 80 30
75 79 29
76 78 28
77 77 27
78 76 26
79 75 25
80 74 24
81 73 23
82 72 22
83 71 21
84 70 20
85 69 19
86 68 18
87 67 17
88 66 16
89 65 15
90 64 14
91 63 13
92 62 12
93 61 11
94 60 10
95 59 9
96 58 8
97 57 7
98 56 6
99 55 5
100 54 4
101 53 3
102 52 2
103 51 1
104 50 0
save
105 49 49
106 4